In [ ]:
import os
import time
import numpy as np
import pandas as pd
import math
from utils import *
import dimod
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

# Sampleset de dwave

En este cuaderno se resolverá un problema formulado en formato QUBO, utilizando el simulador de annealing térmico de Dwave.

El annealing simulado de dwave recibe una matriz $Q$ e *intenta* minimizar la expresión cuadrática una cierta cantidad $n = shots$ de veces. 

Luego, retorna un array de largo $n$ de la forma:

$$[(x_1, energia_1), (x_2, energia_2) , \dots, (x_{n}, energia_n)]$$ 

donde tenemos en $x$ el vector, y en $energia$ su energia asociada. 

> __El vector $x$ de menor energia, será (en teoria) nuestra solucion al problema.__



In [ ]:
def lowest_energy(sampleset):  # Finds the lowest energy solution
    
    # Description: given a full sampleset (tuples of the form (solution, energy) finds the lowest energy SAMPLE.
    # INPUTS:
    # Sampleset: a sampleset of the form list((solution, energy))

    # OUTPUTS:
    # best: a tuple of the form (solution, energy)

    if len(sampleset):
        #energies = np.array(sampleset)[:, 1]  # energias
        energies = [row[1] for row in sampleset]
        index = np.argmin(energies)  # indice de la de menor energia
        ret = sampleset[index]  # solucion de menor energia
        return ret

    else:
        return None

## Funciones de checkeo

Debido a errores cuánticos (incluidos en el simulador), muchos de los posibles $x$ no cumplirán la restricción de peso. 

> Para ello, debemos hacer funciones de checkeo y filtrado del sampleset

Debemos checkear dos cosas:
* Que se respete la restriccion de peso
* Que se prendan correctamente las slacks para representar el $W$

Si las slacks no se prenden correctamente, $W$ queda mal representado en el QUBO, de tal forma que podría pasar que se cumplan las restricciones de peso, pero los valores de energia queden no representativos de la realidad.

Para checkear las slacks:

> Dado un $x$ fijarnos si la suma del peso que cargo + el numero binario representado por las slacks es igual al peso máximo permitido. 

In [ ]:
def wrong_slack(sample, max_weight, weights, num_slack):
    
    # INPUTS:
    # Sample (list): vector x que queremos checkear
    # max_weight (int): peso máximo
    # weights (list): lista de pesos
    # num_slack (int): cantidad de slacks para representar max_weight
    
    # OUTPUTS:
    # False si la solucion es incorrecta
 
    ret = False  # valid solution if true

    # creamos el vector "vect" de slacks teorico, mencionado en el cuaderno de jupyter "qubo.ipynb"

    pesosSlacks = 2 ** (np.arange(num_slack - 1))
    pesosSlacks = np.r_[pesosSlacks, max_weight - sum(pesosSlacks)]
    vect = np.r_[-weights, pesosSlacks] # nos sirve para checkear ACA TAMBIEN CAMBIO SIGNO
    solWeightSlack = sample * vect 


    ws = sum(solWeightSlack) # en ws queda guardado la suma de los pesos cargados + el numero que representan las slacks
    print("ws = ", ws)
    #if abs(ws - max_weight) > 0.001
    if abs(ws) > 0.001: # si la diferencia es casi 0... #DICE MAYOR EN CODIGO ORIGINAL
        ret = True # las slacks se prendieron correctamente
    return ret


def check_weight(sample, max_weight, weights, num_slack):
    # Description:
    # 

    # INPUTS:
    # Sample (list): vector x que queremos checkear
    # max_weight (int): peso máximo
    # weights (list): lista de pesos
    # num_slack (int): cantidad de slacks para representar max_weight
    # OUPUTS:
    # True if solution checks weights inequality, False otherwise

    ret = True
    error_slack = wrong_slack(sample, max_weight, weights, num_slack)
    sample = sample[0:len(sample) - (num_slack)]  # Me quedo con la solucion, ya no me sirven las slacks

    loaded_weight = sum(sample * weights)

    if error_slack:  # si hay error en las slacks, ya descarto de una
        print("weight slack  error")
        ret = False
    else:  # si no hay error en las slacks, me fijo el peso cargado
        if loaded_weight > loaded_weight:
            ret = False
            print("weigh exceeded")

    return ret

# Funciones de simulacion

Esta función instancia un simulador de dwave y retorna un sampleset (recordar, lista de x y energías)  de tamaño "shots".

>La funcion retornará valores diferentes de $x$. A veces no se observan " $n = shots$" soluciones, sino algunas menos. 


In [ ]:
def sendToDwave(qubo, shots=100):
    # Description: functions that solves a particular qubo problem

    # INPUT:
    # qubo: (matrix) representation of the xt*Q*x problem

    # OUTPUT:
    # sampleset: array of tuples of the form (solution, energy) of length "shots" containing posible (but not neccesarily feasible) solutions
    
    tic = time.perf_counter() # for time measuring
    sampleset = dimod.SimulatedAnnealingSampler().sample_qubo(qubo, num_reads=shots)
    sampleset = sampleset.aggregate() # solo agrega soluciones DIFERENTES. 
    sampleset = [(sample, energy) for sample, energy in zip(sampleset.record.sample, sampleset.record.energy)]
    toc = time.perf_counter() # for time measuring
    print(f"Simmulating {shots} instances of annealing took: {(toc-tic)}s")
    # print("Sampleset sin filtrar: ", sampleset)
    return sampleset

# Funcion de filtrado

Dado un sampleset, debemos utilizar las funciones de checkeo definidas anteriormente para descartar soluciones inválidas. 

In [ ]:
def filterKnapSampleset(sampleset,  max_weight, weights,  num_slack):
    # Description:
    # given a FULL SAMPLESET for knapsack problem, filters the invalid samplesets

    # INPUTS:
    # sampleset: raw sampleset (list of tuples) returned from dwave sampler, the structure is: (solution, energy)
    # max_weight: (int) maximum weight
    # weights (list): lista de pesos
    # num_slack (int): cantidad de slacks para representar max_weight
    

    # OUTPUTS:
    # feasibleSamples: sampleset (list tuples) with the valid solutions, the structure is: (validSolution, energy)

    feasibleSamples = []
    cantidadValidas = 0

    i = 0
    for sample, energy in sampleset:
        print("-----------------------")
        print("Checking solution: ", i)

        weightRespected = check_weight(sample, max_weight, weights, num_slack)  # weightFlag

        if weightRespected:  # if sample is valid:
            print("valid solution")
            feasibleSamples.append((sample, energy))
            cantidadValidas = cantidadValidas + 1
        i = i + 1
    return feasibleSamples

# Resolución del problema de knapsack

In [ ]:
# Creamos la misma instancia de knapsack

profits = np.array([18, 15, 10, 10, 18])
weights = np.array([19, 13, 10, 17, 10])
profits = np.array([10, 5, 7, 13])
weights = np.array([4, 3, 5, 7])
data = dict({'weight': weights,'profit': profits})
num_items = len(profits)
max_weight = int(np.floor(num_items / 2 * np.mean(weights)))
max_weight = 12
df = pd.DataFrame(data)
print("-------------------------------------")
print("Choose items from: \n ")
print(df)
print(f"with a max weight of: {max_weight}\n ")
print("-------------------------------------")

In [ ]:
# Calculamos la cantidad de slacks
num_slack = int(math.trunc(np.log2(max_weight))) + 1
print("Cantidad de slacks de peso: ", num_slack)

In [ ]:
# Formulacion del problema
q_objetivo = get_Q_objetivo(profits, num_slack)
print("Matriz q_objetivo: \n", q_objetivo)

In [ ]:
q_peso = get_Q_Peso(max_weight, weights, num_slack)
print("Matriz q_objetivo: \n", q_peso)

In [ ]:
# Armamos el modelo:

alpha = 1 #2/9
qubo = -q_objetivo + alpha * q_peso


print("Dimensiones del problema: ", qubo.shape)



In [ ]:

print("simmulating....")
sampleset = sendToDwave(qubo)
print("Filtering:")
feasibleSampleset = filterKnapSampleset(sampleset,  max_weight, weights,  num_slack)
plt.bar([(str(el[1]+alpha*max_weight**2)) for el in sampleset], [(el[1]+alpha*max_weight**2) for el in sampleset])
plt.xticks(rotation = 90)
plt.show()
bestSample = lowest_energy(feasibleSampleset)
print(sampleset)
solution = bestSample[0]
x_opt = solution[0:num_items] # le saco las slacks



In [ ]:

index_of_chosen = np.where(x_opt == 1)[0] # retorna los indices de los items elegidos.
output_df = df.iloc[index_of_chosen]
print("-------------------------------------")
print("Choosen items are: ")
print(output_df)
print("-------------------------------------")

La solucion fue la misma que en el knapsack lineal.
> Hay que tener en cuenta que se simularon 100 instancias de annealing simulado. La heuristica de cvxpy es mas rapida porque ya esta asociada al problema, aqui simulamos una computadora cuantica, por eso tarda

> Teoricamente, si no hubiese error cuantico, con un solo shot alcancaría, y en una computadora cuantica el tiempo de resolución seria de un tiempo adiabático. 